# Advanced options

We import everything needed for the tutorial.

In [2]:
from causalinflation import InflationProblem, InflationSDP

## How to customize the generating set

We can use the built-in functions to construct a generating set according to several hierarchies. To showcase this, let us first look at a simple scenario of only two parties, Alice and Bob, sharing a quantum state.

In [3]:
InfProb = InflationProblem(dag= {"h1": ["v1", "v2"]},
                           outcomes_per_party=[2, 2],
                           settings_per_party=[2, 2],
                           inflation_level_per_source=[1])
InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)

These are all the measurement operators:

In [4]:
meas = InfSDP.measurements
meas

[[[[A_1_0_0], [A_1_1_0]]], [[[B_1_0_0], [B_1_1_0]]]]

As a reminder, the indices are interpreted as follows: the last two give the measurement setting and measurement output, and the rest represent on which copy of the source the operator is acting. For example, `A_1_1_0` represents the projection onto copy 1 of source 1 with measurement setting $x=1$ and measurement outcome $a=0$.

Now let us look at a few examples of hierarchies. The first is the NPA levels

In [5]:
InfSDP.build_columns(column_specification='local1')
print(InfSDP.generating_monomials_sym)

AttributeError: 'InflationSDP' object has no attribute 'generating_monomials_sym'